In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import pandas as pd

In [13]:
cv=CountVectorizer(ngram_range=(1,3))
text="Even though Thor is a god from Greek mythology, the actor \
portraying the role is originally from Australia"
cv.fit_transform([text])
cv.vocabulary_

{'even': 4,
 'though': 42,
 'thor': 39,
 'is': 17,
 'god': 11,
 'from': 7,
 'greek': 14,
 'mythology': 22,
 'the': 34,
 'actor': 0,
 'portraying': 28,
 'role': 31,
 'originally': 25,
 'australia': 3,
 'even though': 5,
 'though thor': 43,
 'thor is': 40,
 'is god': 18,
 'god from': 12,
 'from greek': 9,
 'greek mythology': 15,
 'mythology the': 23,
 'the actor': 35,
 'actor portraying': 1,
 'portraying the': 29,
 'the role': 37,
 'role is': 32,
 'is originally': 20,
 'originally from': 26,
 'from australia': 8,
 'even though thor': 6,
 'though thor is': 44,
 'thor is god': 41,
 'is god from': 19,
 'god from greek': 13,
 'from greek mythology': 10,
 'greek mythology the': 16,
 'mythology the actor': 24,
 'the actor portraying': 36,
 'actor portraying the': 2,
 'portraying the role': 30,
 'the role is': 38,
 'role is originally': 33,
 'is originally from': 21,
 'originally from australia': 27}

In [9]:
import spacy
nlp=spacy.load("en_core_web_sm")
def preprocess(text):
    doc=nlp(text)
    filtered_tokens=[]
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return ' '.join(filtered_tokens)

In [12]:
preprocess(text)

'Thor god greek mythology actor portray role originally Australia'

# News category processing

In [16]:
df=pd.read_json('/home/siva/Downloads/archive(10)/News_Category_Dataset_v3.json'\
               ,lines=True)
df

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [19]:
df['category'].value_counts()

category
POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATI

In [22]:
df_politics=df[df['category']=='POLITICS'].sample(9800,random_state=5)
df_wellness=df[df['category']=='WELLNESS'].sample(9800,random_state=5)
df_entertainment=df[df['category']=='ENTERTAINMENT'].sample(9800,random_state=5)
df_travel=df[df['category']=='TRAVEL'].sample(9800,random_state=5)
df_style_beauty=df[df['category']=='STYLE & BEAUTY'].sample(9800,random_state=5)
df_sampled=pd.concat([df_politics,df_wellness,df_entertainment,\
                     df_travel,df_style_beauty],axis=0)
df_sampled

,link,headline,category,short_description,authors,date
38355,https://www.huffingtonpost.com/entry/morality-...,"Morality, Race And Chemical Weapons In Syria",POLITICS,Tomahawk Missiles Fired into Syria By Roger Wi...,"Roger Witherspoon, ContributorJournalist",2017-04-10
23115,https://www.huffingtonpost.com/entry/semi-auto...,"Make No Mistake, Semi-Automatic Guns Are Very ...",POLITICS,The AR-15 is able and likely to inflict catast...,"Jon Zal, ContributorWriter and Former U.S. Arm...",2017-10-10
23354,https://www.huffingtonpost.com/entry/president...,President Of Just Some,POLITICS,The image of the president throwing paper towe...,"Susan K Smith, ContributorRev. Dr.",2017-10-07
31004,https://www.huffingtonpost.com/entry/cnn-freed...,CNN's Fourth Of July Tweets Are Obviously Dire...,POLITICS,"Independence Day is all about freedom, includi...",Carla Herreria,2017-07-04
12027,https://www.huffingtonpost.com/entry/josh-kush...,Jared Kushner's Younger Brother Had Vowed To #...,POLITICS,"Josh Kushner, who reportedly donated $50,000 t...",Dominique Mosbergen,2018-03-25
...,...,...,...,...,...,...
152049,https://www.huffingtonpost.com/entry/what-we-c...,What We Can Learn From Dressing,STYLE & BEAUTY,I understand my body now in ways I never did b...,"Sally McGraw, Contributor\nFreelance writer, b...",2013-10-03
167141,https://www.huffingtonpost.com/entry/jessica-a...,Jessica Alba's Style Evolution: From Denim Cut...,STYLE & BEAUTY,Want more? Be sure to check out HuffPost Style...,Michelle Persad,2013-04-26
203138,https://www.huffingtonpost.com/entry/peggy-mof...,Inspired Attire Encouraged,STYLE & BEAUTY,I received my MOCA West Members Opening invita...,"Carrie White, Contributor\nHollywood hairdress...",2012-04-05
207162,https://www.huffingtonpost.com/entry/overworld...,OVERWORLD: Fashion Follies Meet Mario Bros.,STYLE & BEAUTY,"Inspired by Mario Bros., the two mascots of Ni...","Evelyne Politanoff, Contributor\nArts blogger,...",2012-02-23


In [23]:
df_sampled.category.value_counts()

category
POLITICS          9800
WELLNESS          9800
ENTERTAINMENT     9800
TRAVEL            9800
STYLE & BEAUTY    9800
Name: count, dtype: int64

In [30]:
X_train,X_test,y_train,y_test=train_test_split(\
df_sampled['short_description'],df_sampled['category'],test_size=0.2)
clf=Pipeline([('count_vectorizer',CountVectorizer()),
             ('naive_bayes',MultinomialNB())])
clf.fit(X_train,y_train)

Pipeline(steps=[('count_vectorizer', CountVectorizer()),
                ('naive_bayes', MultinomialNB())])

In [31]:
y_pred=clf.predict(X_test)
print(classification_report(y_test,y_pred))

                precision    recall  f1-score   support

 ENTERTAINMENT       0.67      0.61      0.64      1937
      POLITICS       0.82      0.68      0.74      1954
STYLE & BEAUTY       0.81      0.77      0.79      1980
        TRAVEL       0.76      0.75      0.76      1948
      WELLNESS       0.66      0.87      0.75      1981

      accuracy                           0.74      9800
     macro avg       0.74      0.74      0.74      9800
  weighted avg       0.75      0.74      0.74      9800



In [34]:
clf=Pipeline([('count_vectorizer',CountVectorizer(ngram_range=(1,5))),
             ('naive_bayes',MultinomialNB())])
clf.fit(X_train,y_train)

Pipeline(steps=[('count_vectorizer', CountVectorizer(ngram_range=(1, 5))),
                ('naive_bayes', MultinomialNB())])

In [35]:
y_pred=clf.predict(X_test)
print(classification_report(y_test,y_pred))

                precision    recall  f1-score   support

 ENTERTAINMENT       0.70      0.42      0.53      1937
      POLITICS       0.87      0.57      0.69      1954
STYLE & BEAUTY       0.78      0.69      0.73      1980
        TRAVEL       0.68      0.74      0.71      1948
      WELLNESS       0.53      0.93      0.67      1981

      accuracy                           0.67      9800
     macro avg       0.71      0.67      0.67      9800
  weighted avg       0.71      0.67      0.67      9800

